In [ ]:
!pip install -q transformers==4.36.0
!pip install -q bitsandbytes==0.41.3 accelerate==0.25.0
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

In [ ]:
from transformers import pipeline

model_id = "llava-hf/llava-1.5-7b-hf"
pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})

config.json:   0%|          | 0.00/954 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

In [ ]:
from pyngrok import ngrok
authtoken = "set-your-ngrok-auth-token-here"

In [ ]:

ngrok.set_auth_token(authtoken)

In [ ]:
import requests
from pyngrok import ngrok
from typing import List
import uvicorn
import io
import nest_asyncio
from PIL import Image
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from time import sleep
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()
origins = ["*"] 
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# Function to generate text based on image URL and prompt
def image_to_text(image_url):
    # You can modify this part based on your pipeline and prompt logic
    image = Image.open(requests.get(image_url, stream=True).raw)
    prompt = f"USER: <image>\n Tell me what you see? \nASSISTANT:"
    outputs = pipe(image, prompt=prompt, generate_kwargs={"max_new_tokens": 200})
    return  outputs[0]['generated_text'].split('ASSISTANT: ')[1].strip()

@app.post("/generate")
async def generate_text(image_urls: List[str]):
    generated_texts = []

    for image_url in image_urls:
        try:
            text = image_to_text(image_url)
            generated_texts.append(text)
        except Exception as e:
            print(f"Error processing image {image_url}: {str(e)}")

    return JSONResponse(content={"generated_texts": generated_texts})


ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
sleep(0.5)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://e972-34-171-41-8.ngrok-free.app


INFO:     Started server process [673]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     171.49.154.64:0 - "POST /generate?image_url=https://ai.google.dev/static/tutorials/rest_quickstart_files/output_dpu7vQA7-_VR_0.png HTTP/1.1" 200 OK
